In [1]:
# notebooky stuff
%load_ext autoreload
%autoreload 2
from IPython.display import display

import sys 
sys.path.append('../../src/modules')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import eumf_data, eumf_eval, eumf_pipeline
from sklearn import preprocessing, linear_model, model_selection, ensemble, feature_selection
import seaborn as sns


# pandas pretty output
pd.set_option('display.min_rows', 20)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

## Data

### Load all data

In [2]:
# migration rates
df_values = eumf_data.load_registrations_from_csv(impute_missing=True)

# google trends
df_trends = eumf_data.load_trends_from_csv()

countries = eumf_data.get_countries()
keyword_ids = df_trends.columns.levels[0].tolist()

df_gdp = eumf_data.read_gdp()
df_unempl = eumf_data.read_unempl()


### Join data, combine countries

In [3]:
country_combinations = [
    ["GR", "CY"],
    ["LV", "LT", "EE"],
    ["BE", "NL", "LU"],
    ["CZ", "SK"],
    ["SE", "FI", "DK"],
    ["AT", "CH"]
]

panel = df_values.join(df_trends, how="outer")
panel_3m = panel.resample("3M", closed="left").mean()

panel_comb = eumf_data.combine_countries(panel, combinations=country_combinations)
panel_comb_3m = eumf_data.combine_countries(panel_3m, combinations=country_combinations)
df_gdp_comb = eumf_data.combine_countries(df_gdp, combinations=country_combinations)
df_unempl_comb = eumf_data.combine_countries(
    df_unempl, combinations=country_combinations, average=True
)
# note: strictly, unweighted average is wrong for unemployment, but should work in most cases

panel_comb_3m_macro = panel_comb_3m.join(df_gdp_comb).join(df_unempl_comb)

panel_comb_3m_macro["2017":"2018"]


10                                                                                                                        11                                                                                                                                            112                                                                                                               ...   value                                                                                                                                     gdp                                                                                                                                                 unempl                                                                                                       
country     AT+CH  BE+NL+LU  BG  CZ+SK    ES      FR      GB   GR+CY  HR   HU   IE     IT   LV+LT+EE    PL    PT   RO  SE+FI+DK  SI   AT+CH  BE+NL+LU    BG    CZ+SK     ES      FR      GB    GR+CY    HR      HU      IE      IT   LV+LT+EE    PL      PT      RO   SE+FI+DK  SI   AT+CH  BE+NL+LU    BG    CZ+SK      ES      FR      GB    GR+CY     HR      HU      IE      IT   LV+LT+EE    PL    ...     ES        FR        GB      GR+CY       HR        HU       IE        IT     LV+LT+EE     PL        PT        RO     SE+FI+DK    SI     AT+CH   BE+NL+LU    BG    CZ+SK     ES      FR      GB    GR+CY      HR      HU      IE       IT   LV+LT+EE    PL      PT      RO   SE+FI+DK    SI   AT+CH  BE+NL+LU  BG  CZ+SK   ES   FR   GB  GR+CY    HR   HU   IE    IT  LV+LT+EE  PL   PT   RO  SE+FI+DK  SI 
date                                                                                                                                                                                                                                                                                                                                                                                                    ...                                                                                                                                                                                                                                                                                                                                                                                                              
2017-03-31  10.524  16.048   0.0   0.0  26.667  21.857  21.381   0.0  0.0  0.0  0.0  14.095    0.0     6.095  0.0  0.0  21.286   0.0  34.714  60.190    2.190  10.857  38.714  29.810  53.381  12.810  0.000   0.619  11.095  22.048    0.0    11.952  12.714  11.571  26.524   0.0  97.190  178.952  42.238  127.048  33.143  63.333  47.286  29.619  63.095  62.810  59.333  14.905  179.429  39.524  ...  1689.000  1268.333  1080.333  2399.000  4459.000  3747.667  156.000  5255.333  1396.333  12264.667  866.333  18186.333  654.333  420.333  28780.0  42970.0  1510.0  7620.0  5950.0  8450.0  9080.0   9270.0  2590.0  2820.0  14640.0  6800.0  10490.0  2760.0  4520.0  1850.0  33780.0  4790.0  2.85    6.433   6.6  6.00  18.2  9.6  4.6  17.35  12.7  4.3  7.4  11.6   7.333   5.3  9.8  5.3   7.167   7.4
2017-06-30   4.952  14.810   0.0   0.0  21.048  13.952  18.857   0.0  0.0  0.0  0.0  15.286    0.0     6.524  0.0  0.0  27.714   0.0  39.286  61.619   10.619   7.714  24.381  27.952  55.714   8.286  2.333   7.095  20.619  17.286    0.0    13.810  23.143  11.143  32.857   0.0  85.619  168.905  35.190   83.429  23.810  57.857  48.000  36.476  33.048  27.905  62.810  14.286  126.667  21.476  ...  1427.667  1023.333  1049.000  2163.000  4121.333  3637.667  164.667  4699.000  1407.000  12940.000  609.667  19264.000  520.333  365.000  29250.0  44540.0  1800.0  8470.0  6330.0  8560.0  8980.0  10000.0  2990.0  3200.0  14700.0  7140.0  11780.0  2960.0  4790.0  2200.0  35590.0  5260.0  2.70    5.867   6.2  5.75  17.3  9.6  4.4  16.70  11.5  4.3  6.7  11.3   7.667   5.1  9.2  4.9   7.133   6.6
2017-09-30   4.905  16.095   0.0   0.0  12.4

## Experiments

### Common definitions

In [4]:
T_MIN = "2010"
T_MAX = "2019"

T_TEST_MIN = "2014"
T_TEST_MAX = "2014"

cv_default = model_selection.KFold(n_splits=8, shuffle=False)
cv_random = model_selection.KFold(n_splits=5, shuffle=True, random_state=42)

### Compare keywords

In [6]:
### TRAINING

feature_combinations = [[k] for k in keyword_ids]

tuners = []
cv_scores, test_scores = [], []
train_stackeds, test_stackeds, train_unstackeds = [], [], []

params = {}
# params = {
#     "randomforestregressor__max_features": ["auto", "sqrt"],
#     "randomforestregressor__min_samples_leaf": [1, 2, 4, 8],
#     "randomforestregressor__min_samples_split": [2, 4, 8, 16],
# }

for features in feature_combinations:

    labeled = eumf_pipeline.prepare_data(
        panel_comb_3m_macro,
        columns=features,
        lags=[1, 2, 3, 4, 5, 6],
        t_min=T_MIN,
        t_max=T_MAX,
    )
    transformed = eumf_pipeline.transform_data(labeled)
    train, test = eumf_pipeline.split_data(
        transformed, t_test_min=T_TEST_MIN, t_test_max=T_TEST_MAX
    )
    train_stacked, test_stacked = eumf_pipeline.stack_data(train, test)

    train_stackeds.append(train_stacked)
    train_unstackeds.append(train)
    test_stackeds.append(test_stacked)

    tuner = eumf_pipeline.train_reg_model(
        train_stacked,
        reg=ensemble.RandomForestRegressor(random_state=42),
        extra_pipeline_steps=[preprocessing.StandardScaler()],
        params=params,
        scoring=eumf_eval.scorer_rmse,
    )
    tuners.append(tuner)

    cv_score = eumf_eval.score_cv(tuner.best_estimator_, train_stacked, cv=cv_default,)
    cv_scores.append(cv_score)

    test_score = eumf_eval.score_test(tuner.best_estimator_, test_stacked,)
    test_scores.append(test_score)


In [21]:
eumf_eval.agg_multiple_cv_scores(
    cv_scores, feature_combinations, use_quantiles=True
).sort_values(("test_rmse", 0.5), ascending=False)[["test_r2_mod", "test_rmse"]]


test_r2_mod               test_rmse              
            0.25     0.50   0.75     0.25    0.50   0.75 
['19']     -0.055    0.207  0.439   -0.162  -0.112 -0.096
['112']    -0.573   -0.284  0.028   -0.218  -0.139 -0.132
['24']     -0.646   -0.349  0.147   -0.214  -0.142 -0.119
['12']     -0.825   -0.241  0.136   -0.211  -0.143 -0.123
['21']     -0.934   -0.488  0.099   -0.219  -0.149 -0.130
['119']    -0.998   -0.387  0.176   -0.199  -0.149 -0.137
['28']     -1.064   -0.248  0.187   -0.206  -0.149 -0.130
['20']     -0.999   -0.489  0.102   -0.218  -0.150 -0.136
['48']     -1.088   -0.413  0.133   -0.215  -0.150 -0.132
['32']     -1.103   -0.475  0.070   -0.221  -0.151 -0.134
['14']     -0.924   -0.416  0.032   -0.230  -0.152 -0.126
['16']     -1.092   -0.397  0.093   -0.212  -0.153 -0.133
['123']    -0.447   -0.174  0.078   -0.228  -0.153 -0.126
['125']    -0.983   -0.517  0.172   -0.208  -0.153 -0.128
['2']      -1.097   -0.481  0.115   -0.218  -0.153 -0.136
['115']    -0.705   -0.413  0.170   -0.198  -0.153 -0.124
['49']     -0.816   -0.401  0.096   -0.226  -0.154 -0.122
['44']     -0.951   -0.451  0.059   -0.224  -0.154 -0.130
['121']    -1.369   -0.793  0.107   -0.219  -0.154 -0.150
['15']     -1.171   -0.439  0.141   -0.212  -0.155 -0.134
['114']    -0.675   -0.397  0.219   -0.204  -0.156 -0.124
['22']     -1.249   -0.651  0.026   -0.228  -0.156 -0.139
['56']     -1.121   -0.702  0.057   -0.225  -0.156 -0.138
['54']     -1.070   -0.615  0.094   -0.217  -0.157 -0.135
['52']     -1.002   -0.550  0.134   -0.214  -0.157 -0.128
['11']     -0.993   -0.708  0.056   -0.223  -0.159 -0.130
['33']     -1.140   -0.459  0.098   -0.219  -0.159 -0.141
['118']    -0.898   -0.213  0.180   -0.207  -0.159 -0.138
['18']     -1.132   -0.675  0.099   -0.218  -0.160 -0.140
['43']     -1.104   -0.622  0.109   -0.217  -0.161 -0.135
['117']    -0.855   -0.622  0.123   -0.214  -0.161 -0.129
['38']     -1.328   -0.550  0.101   -0.217  -0.161 -0.138
['59']     -1.133   -0.560  0.109   -0.218  -0.162 -0.135
['60']     -1.081   -0.800  0.058   -0.223  -0.162 -0.139
['42']     -1.133   -0.667  0.104   -0.218  -0.163 -0.142
['51']     -1.358   -0.668  0.045   -0.225  -0.163 -0.142
['26']     -1.145   -0.648  0.109   -0.214  -0.164 -0.134
['57']     -1.140   -0.615  0.060   -0.224  -0.164 -0.132
['39']     -0.924   -0.315  0.197   -0.206  -0.165 -0.131
['10']     -1.214   -0.658  0.072   -0.222  -0.165 -0.139
['124']    -1.203   -0.592  0.102   -0.219  -0.165 -0.128
['113']    -1.263   -0.803  0.034   -0.219  -0.169 -0.149
['37']     -1.256   -0.866  0.119   -0.216  -0.171 -0.139
['17']     -1.223   -0.643  0.072   -0.219  -0.172 -0.138
['41']     -1.241   -0.621  0.076   -0.223  -0.174 -0.131
['122']    -1.035   -0.687  0.240   -0.198  -0.179 -0.141
['58']     -1.164   -0.206  0.129   -0.216  -0.183 -0.126
['55']     -2.052   -1.172 -0.062   -0.232  -0.192 -0.147

### Select k best

In [28]:
### TRAINING

feature_combinations = [
    ["19"],
    ["19", "112"],
    ["19", "112", "24"],
    ["19", "112", "24", "12"],
    ["19", "112", "24", "12", "21"],
    ["19", "112", "24", "12", "21", "119"],
    ["19", "112", "24", "12", "21", "119", "28"],
    ["19", "112", "24", "12", "21", "119", "28", "20"],
    ["19", "112", "24", "12", "21", "119", "28", "20", "48"],
]

tuners = []
cv_scores, test_scores = [], []
train_stackeds, test_stackeds, train_unstackeds = [], [], []

params = {}
# params = {
#     "randomforestregressor__max_features": ["auto", "sqrt"],
#     "randomforestregressor__min_samples_leaf": [1, 2, 4, 8],
#     "randomforestregressor__min_samples_split": [2, 4, 8, 16],
# }

for features in feature_combinations:

    labeled = eumf_pipeline.prepare_data(
        panel_comb_3m_macro,
        columns=features,
        lags=[1, 2, 3, 4, 5, 6],
        t_min=T_MIN,
        t_max=T_MAX,
    )
    transformed = eumf_pipeline.transform_data(labeled)
    train, test = eumf_pipeline.split_data(
        transformed, t_test_min=T_TEST_MIN, t_test_max=T_TEST_MAX
    )
    train_stacked, test_stacked = eumf_pipeline.stack_data(train, test)

    train_stackeds.append(train_stacked)
    train_unstackeds.append(train)
    test_stackeds.append(test_stacked)

    tuner = eumf_pipeline.train_reg_model(
        train_stacked,
        reg=ensemble.RandomForestRegressor(random_state=42),
        extra_pipeline_steps=[preprocessing.StandardScaler()],
        params=params,
        scoring=eumf_eval.scorer_rmse,
    )
    tuners.append(tuner)

    cv_score = eumf_eval.score_cv(tuner.best_estimator_, train_stacked, cv=cv_default,)
    cv_scores.append(cv_score)

    test_score = eumf_eval.score_test(tuner.best_estimator_, test_stacked,)
    test_scores.append(test_score)


In [29]:
eumf_eval.agg_multiple_cv_scores(
    cv_scores, feature_combinations, use_quantiles=True
).sort_values(("test_rmse", 0.5), ascending=False)[["test_rmse", "test_r2_mod"]]


test_rmse               test_r2_mod              
                                                      0.25    0.50   0.75      0.25     0.50   0.75 
['19']                                               -0.162  -0.112 -0.096    -0.055    0.207  0.439
['19', '112', '24']                                  -0.170  -0.113 -0.093    -0.064    0.196  0.463
['19', '112']                                        -0.164  -0.113 -0.098    -0.072    0.144  0.486
['19', '112', '24', '12', '21', '119', '28', '2...   -0.161  -0.113 -0.096    -0.070    0.118  0.376
['19', '112', '24', '12', '21', '119', '28', '20']   -0.161  -0.113 -0.096    -0.069    0.141  0.375
['19', '112', '24', '12', '21', '119', '28']         -0.163  -0.113 -0.097    -0.077    0.138  0.373
['19', '112', '24', '12']                            -0.166  -0.114 -0.093    -0.066    0.185  0.467
['19', '112', '24', '12', '21', '119']               -0.162  -0.114 -0.097    -0.094    0.110  0.363
['19', '112', '24', '12', '21']                      -0.166  -0.115 -0.091    -0.105    0.220  0.438